# 1. Класс Vector

Реализуйте класс `Vector`, допускающий следующие операции:
1. Конструктор, который принимает на вход итерируемый объект. Данные должны храниться в виде стандартного объекта `array` (не `numpy.array`!) типа `float64`;

2. Класс должен быть итерируемым (реализовать метод `__iter__`). В частности, должен работать следующий код

```
>>> v1 = Vector([3, 4, 5])
>>> x, y, z = v1
>>> x, y, z
(3.0, 4.0, 5.0)
```

3. Корректную операцию сравнения `==` (реализовать метод `__eq__`). Два объекта будем считать равными, если совпадает длина и равны все элементы

```
>>> v1 = Vector([3, 4, 5])
>>> v2 = Vector([3.0, 4.0, 5.0])
>>> v3 = Vector([3, 4])
>>> v1 == v2
True
>>> v1 == v3
False
```

4. Класс должен уметь возвращать в разумном виде свое текстовое представление (реализовать метод `__repr__`):

```
>>> v1 = Vector([3, 4, 5])
>>> v1
Vector((3, 4, 5))
```
При правильно реализованной операции равенства должен работать следующий код:

```
>>> v2 = eval(repr(v1))
>>> v1 == v2
True
```

5. Класс должен уметь корректно приводить объект к строчному типу, в частности, работать с функцией `print` (реализовать метод `__str__`):

```
>>> v1 = Vector([3, 4, 5])
>>> print(v1)
(3.0, 4.0, 5.0)
```

6. Класс должен уметь возвращать свою длину методом `len` (реализовать `__len__`) и допускать индексацию, в том числе с помощью срезов (реализовать `__getitem__`). При этом срез должен возвращать (как и все стандартные типы в питоне) копию объекта.

```
>>> v1 = Vector([3, 4, 5])
>>> len(v1)
3
>>> v1[1]
4.0
>>> v1[-1]
5.0
>>> v1[3.14]
TypeError: Vector indices must be integers
>>> v2 = v1
>>> v3 = v1[:]
>>> v1 is v2
True
>>> v1 is v3
False
```

7. Операцию получения евклидовой длины и приведение к типу `bool`. При приведении к типу `bool` нулевой вектор приводится к `False`, все остальные вектора к `True` (`__abs__`, `__bool__`):

```
>>> v1 = Vector([3.0, 4.0])
>>> abs(v1)
5.0
>>> v2 = Vector([0.0, 0.0, 0.0])
>>> bool(v1)
False
>>> bool(v2)
True
```

8. Операцию поэлементного сложения векторов (`__add__`) и умножения на константу. Умножать на константу должна быть возможность как справа, так и слева (`__mul__` и `__rmul__`):

```
>>> v1 = Vector([3, 4, 5])
>>> v2 = Vector([6, 7, 8])
>>> v1 + v2
Vector([9.0, 11.0, 13.0])
>>> v1 * 2
Vector([6.0, 8.0, 10.0])
>>> 2 * v1
Vector([6.0, 8.0, 10.0])
>>> v1 + 2
TypeError: unsupported operand type(s) for +: 'Vector' and 'int'
```
Для получения корректной ошибки метод должен возвращать `return NotImplemented`.

9. Векторы длины не больше 4 должны допускать возможность индексации с помощью `v1.x, v1.y, v1.z, v1.t`:

```
>>> v1 = Vector(range(4))
>>> v1.x
0.0
>>> v1.y, v1.z, v1.t
(1.0, 2.0, 3.0)
>>> v2 = Vector(range(3))
>>> v2.x, v2.y, v2.z
(0.0, 1.0, 2.0)
>>> v2.t
AttributeError: 'Vector' object has no attribute 't'
```
Для этого реализуйте метод `__getattr__`, который в случае некорректного атрибута возвращает исключение `raise AttributeError`.

10. Реализуйте возможность использовать вектор в виде ключей словаря. Для этого реализуйте метод хеширования вектора (`__hash__`). Как один из вариантов построения хеша можно реализовать операцию `xor` от хешей всех элементов вектора. Возможно, вам пригодятся для этого функции `functools.reduce` и `operator.xor`, если вам надоело писать циклы `for`:

```
>>> v1 = Vector([0, 1, 2])
>>> v2 = Vector([3, 4, 5])
>>> d = {}
>>> d[v1] = 'abc'
>>> d[v2] = 'def'
>>> d
{Vector([0.0, 1.0, 2.0]): 'abc', Vector([3.0, 4.0, 5.0]): 'def'}
```

In [28]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import math
import functools
import operator

class Vector(object):
    def __init__(self, input):
        self.storage = [i for i in input]
#         self.cnt = 0
        self.size = len(self.storage)
        
    def __repr__(self):
        return "Vector((" + ', '.join(map(str, self.storage)) + "))"
    
    def __str__(self):
        return "(" + ', '.join(map(str, map(float, self.storage))) + ")"
    
#     def __next__(self):
#         if (self.cnt >= self.size):
#             raise StopIteration()
#         else:
#             self.cnt += 1
#             return self.storage[self.cnt - 1]
        
    def __iter__(self):
#         self.cnt = 0
        return iter(self.storage)
    
    def __eq__(self, right):
        return (len(self.storage) == len(right.storage)) and len(list(filter(lambda x: x in self.storage, right.storage))) == len(self.storage) 
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, i):
        if isinstance(i, int):
            return self.storage[i]
        if isinstance(i, slice):
            return Vector(self.storage[i])
        raise TypeError("Vector indices must be integers")
    
    def __abs__(self):
        return math.sqrt(sum(x**2 for x in self.storage))
    
    def __bool__(self):
        return any(self.storage)
    
    def __add__(self, right):
        if type(right) is not Vector:
            return NotImplemented
        return Vector([float(x + y) for x, y in zip(self.storage, right.storage)])
    
    def __mul__(self, right):
        return Vector(float(x * right) for x in self.storage)
        
    def __rmul__(self, left):
        return Vector(float(x * left) for x in self.storage)
        
    def __getattr__(self, item):
        if (self.size > 4 or self.size == 0):
            raise AttributeError
         
        if (self.size == 1) and (item != "x"):
            raise AttributeError
        if (self.size == 2) and (item != "x") and (item != "y"):
            raise AttributeError
        if (self.size == 3) and (item != "x") and (item != "y") and (item != "z"):
            raise AttributeError
        if (self.size == 4) and (item != "x") and (item != "y") and (item != "z") and (item != "t"):
            raise AttributeError
        
        d = {'x': 0, 'y': 1, 'z': 2, 't': 3}
            
        return float(self.storage[d[item]])
    
    def __hash__(self):
        return functools.reduce(operator.xor, map(hash, self.storage))

# 2. Класс Key2StrDict

Реализуйте класс `Key2StrDict`, работающий как словарь, но преобразующий все поступающие ключи в строки. Если ключ отсутствует в словаре, объект должен возвращать число `0`. Для реализации может быть удобно (но не необходимо) отнаследоваться от класса `collections.UserDict`. В этом случае достаточно реализовать методы `__missing__`, `__contains__` и `__setitem__`.

```
>>> d = Key2StrDict([('abc', 2), ((1,), 0)])
>>> data = [2, 3.14, frozenset([1, 2, 2]), (1, 5, 6, 8), 2, 2, frozenset([2, 1, 1])]
>>> for t in data:
...     d[t] += 1
>>> d
{'abc': 2, '(1,)': 0, '2': 3, '3.14': 1, 'frozenset({1, 2})': 2, '(1, 5, 6, 8)': 1}
```

In [3]:
import collections
class Key2StrDict(collections.UserDict):
    def __missing__(self, key):
        return 0

    def __contains__(self, key):
        return self.data.get(str(key))
    
    def __setitem__(self, key, value):
        self.data[str(key)] = value

d = Key2StrDict([('abc', 2), ((1,), 0)])

False

# 3. Измерение времени работы

В этом задании нам предстоит решать задачу поиска объекта в множестве и измерять время работы этой операции. Искать мы будем вещественные числа в множестве вещественных чисел. Для этого

1. Сгенерируйте множество чисел, которые мы будем искать (обозначим его `t`, но в коде вы можете назвать его как хотите) и множество чисел, среди которых мы будем искать (`s`). Размер множества `t` должен быть равен 1,000, а размер множества `s` будет варьироваться. Пересечение множеств `t` и `s` должно состоять ровно из 500 элементов. При этом элементы должны быть случайным образом разбросаны, то есть, например, элементы из пересечение не должны лежать в `s` первыми.

2. Реализуйте поиск с помощью списков. Например, таким образом

```
found = 0
for n in t:
    if n in s:
        found += 1
```

3. Реализуйте поиск с помощью словаря (`dict`). Для создания словаря с ключами без значений стоит использовать метод `fromkeys()`.

4. Реализуйте поиск с помощью множеств (`set`), аналогичный пункту 2.

5. Реализуйте поиск с помощью множеств (`set`), используя пересечения

```
found = len(t & s)
```

6. Варьируя размер множества `s` от $10^3$ до $10^7$ с шагом в 10 раз проведите измерения времени для всех 4 описанных методов поиска. Для каждого из методов измерьте его время работы и во сколько раз произошло замедление метода относительно наименьшего размера (1,000). Времена на создание структур данных считать не нужно, только время поиска. Для измерения времени используйте магическую команду `%timeit` Постройте таблицу с результатами (желательно в `pandas`).

7. Сделайте выводы об асимптотической сложности. Выводы сделайте обязательно, но устно и просто их запомните, не надо писать здесь сочинения, пожалуйста.

In [27]:
import random
import pandas as pd

def myfind_1(s, t):
    found = 0
    for n in t:
        if n in s:
            found += 1
    return found

def myfind_2(d, t):
    found = 0
    for n in t:
        if n in d:
            found += 1         
    return found

def myfind_3(s_set, t_set):
    found = 0
    for n in t_set:
        if n in s_set:
            found += 1         
    return found

def myfind_4(s_set, t_set):
    return len(t_set & s_set)

result = list(map(lambda x: 10 ** x, range(3, 8)))

index = 0
time_1 = [0 for i in range(5)]
time_2 = [0 for i in range(5)]
time_3 = [0 for i in range(5)]
time_4 = [0 for i in range(5)]

for size in result:
    t = random.sample(range(1, 5000), 1000)

    s = t[:500]
    s += random.sample(range(6000, 6000 + size + 1), size)
    random.shuffle(s)
    
    t = list(map(float, t))
    s = list(map(float, s))
    
    #FIRST
    print("\nFirst method")
    time_t = %timeit -o founded = myfind_1(s, t)
    time_1[index] = time_t.average
    
    #SECOND
    print("\nSecond method")
    d = dict.fromkeys(s)
    time_t = %timeit -o founded = myfind_2(d, t)
    time_2[index] = time_t.average
    

    #THIRD
    print("\nThird method")
    s_set = set(s)
    t_set = set(t)
    time_t = %timeit -o founded = myfind_3(s_set, t_set)
    time_3[index] = time_t.average
    
    #FORTH
    print("\nForth method")
    s_set = set(s)
    t_set = set(t)
    time_t = %timeit -o founded = myfind_4(s_set, t_set)
    time_4[index] = time_t.average

    index += 1
    
mydata = {
    'Algothime': [10**3, 10**4, 10**5, 10**6, 10**7],
    'First': time_1,
    'Second': time_2,
    'Third': time_3,
    'Forth': time_4,
}

df = pd.DataFrame(data = mydata)
df


First method
28.8 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Second method
40 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Third method
48 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Forth method
18.9 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

First method
191 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Second method
46.6 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Third method
47.2 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Forth method
19.2 µs ± 55.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

First method
1.83 s ± 3.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Second method
46.8 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Third method
45.8 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Forth method
18.4 µs ± 218 ns per loo

KeyboardInterrupt: 

# 4. Какой-то алгоритм

Реализуйте алгоритм, который вам выдали на С++, на питоне. Пользоваться можно только стандартной библиотекой питона, использовать `numpy` пока нельзя. К реализации обязательно должна прилагаться проверка корректности работы. Например, если вы реализуете $QR$-разложение, то нужно проверить, что
1. $R$ - верхняя треугольная;
2. $Q$ - ортогональная;
3. $A = QR$.

Получившийся код нужно прогнать через профилировщик.

In [4]:
import random
import numpy as np

def mtx_sum(mtx_A, mtx_B, split_index, multiplier = 1):
    mtx_res = [[0 for _ in range(split_index)] for _ in range(split_index)]
    
    for i in range(split_index):
        for j in range(split_index):
            mtx_res[i][j] = mtx_A[i][j] + (multiplier * mtx_B[i][j])
            
    return mtx_res

def mtx_mult(mtx_A, mtx_B):
    col_1 = len(mtx_A[0])
    row_1 = len(mtx_A)
    col_2 = len(mtx_B[0])
    row_2 = len(mtx_B)
    
    result_matrix = [[0 for _ in range(col_2)] for _ in range(row_1)]
    
    if col_1 == 1:
        result_matrix[0][0] = mtx_A[0][0] * mtx_B[0][0]
    else:
        split_index = col_1 // 2

        a_11 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        a_12 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        a_21 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        a_22 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        b_11 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        b_12 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        b_21 = [[0 for _ in range(split_index)] for _ in range(split_index)]
        b_22 = [[0 for _ in range(split_index)] for _ in range(split_index)]

        for i in range(split_index):
            for j in range(split_index):
                a_11[i][j] = mtx_A[i][j]
                a_12[i][j] = mtx_A[i][j + split_index]
                a_21[i][j] = mtx_A[split_index + i][j]
                a_22[i][j] = mtx_A[i + split_index][j + split_index]
                b_11[i][j] = mtx_B[i][j]
                b_12[i][j] = mtx_B[i][j + split_index]
                b_21[i][j] = mtx_B[split_index + i][j]
                b_22[i][j] = mtx_B[i + split_index][j + split_index]

                
                
        D = mtx_mult(mtx_sum(a_11, a_22, split_index), mtx_sum(b_11, b_22, split_index))
        D_1 = mtx_mult(mtx_sum(a_12, a_22, split_index, -1), mtx_sum(b_21, b_22, split_index))
        D_2 = mtx_mult(mtx_sum(a_21, a_11, split_index, -1), mtx_sum(b_11, b_12, split_index))
        H_2 = mtx_mult(mtx_sum(a_21, a_22, split_index), b_11)
        H_1 = mtx_mult(mtx_sum(a_11, a_12, split_index), b_22)
        V_1 = mtx_mult(a_22, mtx_sum(b_21, b_11, split_index, -1))
        V_2 = mtx_mult(a_11, mtx_sum(b_12, b_22, split_index, -1))


        result_matrix_00 = mtx_sum(mtx_sum(mtx_sum(D, D_1, split_index), V_1, split_index), H_1, split_index, -1)
        result_matrix_01 = mtx_sum(V_2, H_1, split_index)
        result_matrix_10 = mtx_sum(V_1, H_2, split_index)
        result_matrix_11 = mtx_sum(mtx_sum(mtx_sum(D, D_2, split_index), V_2, split_index), H_2, split_index, -1)

        for i in range(split_index):
            for j in range(split_index):
                result_matrix[i][j] = result_matrix_00[i][j]
                result_matrix[i][j + split_index] = result_matrix_01[i][j]
                result_matrix[split_index + i][j] = result_matrix_10[i][j]
                result_matrix[i + split_index][j + split_index] = result_matrix_11[i][j]

    return result_matrix

def Generator(size):
    return [random.sample(range(1, size * 2), size) for _ in range(size)]


size = 32
mtx_A = Generator(size)
mtx_B = Generator(size)

In [5]:
%load_ext line_profiler
%lprun -f mtx_mult mtx_mult(mtx_A, mtx_B)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [6]:
print((np.matmul(mtx_A, mtx_B) == mtx_mult(mtx_A, mtx_B)).all())

True
